This code is for Example 4 on the L shaper domain $(0,3) \times (0,3)$ \ $[2,3) \times [2,3)$ when $A = 25(-y, x)$

# This section is to calculate the eigenpairs for H(A)

In [2]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
from pyeigfeast import NGvecs, SpectralProjNG
import time
start1 = time.time()
a = 25
A = CF((-a*y, a*x))

alpha = 90
side = 1
domain = WorkPlane().MoveTo(0, 0).Line(side*3).Rotate(alpha).Line(side).Rotate(180-alpha).Line(side*2). \
    Rotate(-(180-alpha)).Line(side*2).Rotate(180-alpha).Line(side).Rotate(alpha).Line(side*3).Face()
geo = OCCGeometry(domain, dim = 2)
mesh = Mesh(geo.GenerateMesh(maxh=0.05))
X = H1(mesh, order=3, complex=True)
psi, phi = X.TnT()
a3 = BilinearForm(X)
a3 += grad(psi) * grad(phi) * dx +  1j * InnerProduct(A,grad(phi)) * psi * dx - 1j * InnerProduct(A , grad(psi)) * phi * dx \
       + InnerProduct(A,A)* psi * phi *dx
b3 = BilinearForm(X)
b3 += psi * phi * dx
a3.Assemble()
b3.Assemble()

seed =1
npts=10
nspan=8
within=None
rhoinv=0.0
quadrule='circ_trapez_shift'
verbose=True
# Make the spectral projector object, change radius and center values for the searching range of eigenvalues.
# When h=0.01, radius = 3, center = 28; When h=0.03, radius = 3, center = 28; When h=0.05, radius = 5, center = 30
P = SpectralProjNG(X,
                   a3.mat,
                   b3.mat,
                   radius=5,
                   center=30,
                   npts=npts,
                   within=within,
                   rhoinv=rhoinv,
                   quadrule=quadrule,
                   inverse=None)
Y = NGvecs(X, nspan, M=b3.mat)
Y.setrandom(seed=seed)
start2 = time.time()
lam, Y, history, _ = P.feast(Y)
end = time.time()
print("Total Time", end - start1, "FEAST Time", end - start2)
y = Y.gridfun()
Draw(y)


SpectralProj: Setting shifted trapezoidal rule quadrature on circular contour
SpectralProj: Radius=5, Center=30+0j

SpectralProjNG: Computing resolvents using umfpack
SpectralProjNG:   Factorizing at z = +34.755 +1.545j
SpectralProjNG:   Factorizing at z = +32.939 +4.045j
SpectralProjNG:   Factorizing at z = +30.000 +5.000j
SpectralProjNG:   Factorizing at z = +27.061 +4.045j
SpectralProjNG:   Factorizing at z = +25.245 +1.545j
SpectralProjNG:   Factorizing at z = +25.245 -1.545j
SpectralProjNG:   Factorizing at z = +27.061 -4.045j
SpectralProjNG:   Factorizing at z = +30.000 -5.000j
SpectralProjNG:   Factorizing at z = +32.939 -4.045j
SpectralProjNG:   Factorizing at z = +34.755 -1.545j

=========== Starting FEAST iterations ===========
Trying with 8 vectors:

 ITERATION 1 with 8 vectors
   Real part of computed eigenvalues:
   [25.49674462 30.7528443  31.23993477 32.12242908 33.42708917 33.84964156
 34.9566825  37.72860372]
   Relative Hausdorff distance from prior iterate: 3.333e+9

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

Calculate the degree of freedom 

In [3]:
X.ndof 

20845

Calculate the $L^2$ norm of $\nabla \psi$ and $A\psi$

In [4]:
for i in range(0,6):
    psi = GridFunction(X)
    psi.Set(y.MDComponent(i))
    print(sqrt(Integrate(grad(psi)*Conj(grad(psi)), mesh)), sqrt(Integrate(A*psi*Conj(A*psi), mesh)))

(9.211090563578034+0j) (9.211219673862727+0j)
(31.889400706161656+0j) (31.901384754019507+0j)
(35.491759990989294+0j) (35.50873196104633+0j)
(43.47183704823636+0j) (43.528400183392336+0j)
(47.236939082260314+0j) (47.320432405471614+0j)
(36.99550992101083+0j) (37.03132757295606+0j)


# This section is to calculate the eigenpairs for H(F)

In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
from pyeigfeast import NGvecs, SpectralProjNG
import time
start1 = time.time()
alpha = 90
side = 1 

domain = WorkPlane().MoveTo(0, 0).Line(side*3).Rotate(alpha).Line(side).Rotate(180-alpha).Line(side*2). \
    Rotate(-(180-alpha)).Line(side*2).Rotate(180-alpha).Line(side).Rotate(alpha).Line(side*3).Face()
geo = OCCGeometry(domain, dim = 2)
mesh = Mesh(geo.GenerateMesh(maxh=0.05))
a = 25
A = CF((-a*y, a*x))

X = H1(mesh, order=3)
N = NumberSpace(mesh)
fesm = X*N

u, lam = fesm.TrialFunction()
v, c = fesm.TestFunction()
gfm = GridFunction(fesm)  # solution
gfu, gflam = gfm.components

a = BilinearForm(fesm)
a += lam*v*dx + grad(u)*grad(v)*dx + u*c*dx
a.Assemble()
f = LinearForm(A*grad(v)*dx).Assemble()

gfm.vec.data = a.mat.Inverse() * f.vec
F = A - grad(gfu)

Xeig = H1(mesh, order=3, complex = True)
psi, phi = Xeig.TnT()
a2 = BilinearForm(Xeig)
a2 += grad(psi) * grad(phi) * dx + 1j *  InnerProduct(F,grad(phi)) * psi * dx - 1j * InnerProduct(F , grad(psi)) * phi * dx\
    +  InnerProduct(F, F)* psi * phi *dx
b2 = BilinearForm(Xeig)

b2 += psi * phi * dx
a2.Assemble()
b2.Assemble()

seed =1
npts=10
nspan= 10
within=None
rhoinv=0.0
quadrule='circ_trapez_shift'
verbose=True
# Make the spectral projector object, radius = 4, center = 28 for all h
P = SpectralProjNG(Xeig,
                   a2.mat,
                   b2.mat,
                   radius=4,
                   center=28,
                   npts=npts,
                   within=within,
                   rhoinv=rhoinv,
                   quadrule=quadrule,
                   inverse=None)
Y = NGvecs(Xeig, nspan, M=b2.mat)
Y.setrandom(seed=seed)
start2 = time.time()
lam, Y, history, _ = P.feast(Y)
end = time.time()
print("Total Time", end - start1, "FEAST Time", end - start2)
y1 = Y.gridfun()
Draw(y1)


SpectralProj: Setting shifted trapezoidal rule quadrature on circular contour
SpectralProj: Radius=4, Center=28+0j

SpectralProjNG: Computing resolvents using umfpack
SpectralProjNG:   Factorizing at z = +31.804 +1.236j
SpectralProjNG:   Factorizing at z = +30.351 +3.236j
SpectralProjNG:   Factorizing at z = +28.000 +4.000j
SpectralProjNG:   Factorizing at z = +25.649 +3.236j
SpectralProjNG:   Factorizing at z = +24.196 +1.236j
SpectralProjNG:   Factorizing at z = +24.196 -1.236j
SpectralProjNG:   Factorizing at z = +25.649 -3.236j
SpectralProjNG:   Factorizing at z = +28.000 -4.000j
SpectralProjNG:   Factorizing at z = +30.351 -3.236j
SpectralProjNG:   Factorizing at z = +31.804 -1.236j

=========== Starting FEAST iterations ===========
Trying with 10 vectors:

 ITERATION 1 with 10 vectors
   Real part of computed eigenvalues:
   [24.62756093 24.6643055  25.50322809 26.8331355  26.84884798 30.37423483
 30.66220534 30.8999435  31.43572596 32.73039521]
   Relative Hausdorff distance fr

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

Calculate the $L^2$ norm of $\nabla \phi$ and $F\phi$

In [2]:
for i in range(0,6):
    phi = GridFunction(Xeig)
    phi.Set(y1.MDComponent(i))
    print(sqrt(Integrate(grad(phi)*Conj(grad(phi)), mesh)), sqrt(Integrate(F*phi*Conj(F*phi), mesh)))

(11.837157579794106+0j) (11.882442276809583+0j)
(11.836179985523676+0j) (11.88148466789254+0j)
(14.413314531706877+0j) (14.413425116253624+0j)
(12.302921348210711+0j) (12.221508369812287+0j)
(12.296587758953548+0j) (12.212956816252+0j)
(20.025638099013364+0j) (20.008943538248513+0j)
